# ⚽ Simulation Ligue des Champions – Modèle simple de prédiction
Ce notebook propose une simulation simplifiée de la Ligue des Champions.

**Objectifs :**
- Créer un dataset d'équipes avec un indice de performance
- Simuler des confrontations à élimination directe
- Estimer la probabilité pour chaque club d’atteindre les différentes phases
- Identifier les favoris et les outsiders

> 🔁 Les données sont simulées mais structurées comme un vrai cas d’étude.

## 1. Import des bibliothèques

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## 2. Création d'un dataset d'équipes
On crée une liste d’équipes avec un indice de performance.


In [ ]:
teams = [
    'Real Madrid','FC Barcelona','Bayern Munich','Manchester City','Liverpool',
    'Chelsea','Paris Saint-Germain','Juventus','Atletico Madrid','Borussia Dortmund',
    'Inter Milan','AC Milan','Arsenal','Napoli','RB Leipzig','Tottenham'
]

performance_index = [95,92,93,94,90,88,91,87,86,85,84,83,82,81,80,79]

clubs = pd.DataFrame({
    'Team': teams,
    'PerfIndex': performance_index
})

clubs

## 3. Fonction de simulation de match
Probabilité de victoire basée sur les indices de performance.

In [ ]:
def simulate_match(team_a, team_b, df):
    perf_a = df.loc[df.Team == team_a, 'PerfIndex'].values[0]
    perf_b = df.loc[df.Team == team_b, 'PerfIndex'].values[0]
    p_a = perf_a / (perf_a + perf_b)
    return team_a if random.random() < p_a else team_b

## 4. Simulation d'un tour à élimination directe

In [ ]:
def simulate_round(teams_list, clubs_df, round_name='Round'):
    random.shuffle(teams_list)
    winners = []
    print(f"\n=== {round_name} ===")
    for i in range(0, len(teams_list), 2):
        a, b = teams_list[i], teams_list[i+1]
        w = simulate_match(a, b, clubs_df)
        print(f"{a} vs {b} → {w}")
        winners.append(w)
    return winners

In [ ]:
def simulate_knockout(clubs_df):
    teams_round = clubs_df.Team.tolist()
    r16 = simulate_round(teams_round, clubs_df, 'Huitièmes de finale')
    qf = simulate_round(r16, clubs_df, 'Quarts de finale')
    sf = simulate_round(qf, clubs_df, 'Demi-finales')
    final = simulate_round(sf, clubs_df, 'Finale')
    winner = final[0]
    print(f"\n🏆 Vainqueur : {winner}\n")
    return winner

## 5. Exemple d'une simulation

In [ ]:
winner_example = simulate_knockout(clubs)

## 6. Simulation Monte Carlo (1000 simulations)
Estimations des probabilités de victoire.

In [ ]:
N = 1000
winners = [simulate_knockout(clubs) for _ in range(N)]
winners_series = pd.Series(winners)
win_counts = winners_series.value_counts().reindex(clubs.Team).fillna(0)
win_probs = win_counts / N
win_probs

### Visualisation

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=win_probs.index, y=win_probs.values)
plt.xticks(rotation=45)
plt.title('Probabilité de remporter la LDC')
plt.show()

## 7. Synthèse des résultats

In [ ]:
clubs_results = clubs.copy()
clubs_results['WinProb'] = win_probs.values
clubs_results.sort_values('WinProb', ascending=False)

## 8. Analyse et insights
- Les clubs avec le plus fort indice sont logiquement favoris.
- La variabilité des simulations laisse une place aux surprises.
- On peut enrichir ce modèle avec : forme récente, buts, domicile/extérieur, blessures…